## Telegram data scraping
### Import Required Modules

In [1]:
# Import required modules
import os
import sys
import asyncio
import logging
import pandas as pd
from dotenv import load_dotenv
from telethon import TelegramClient

import nest_asyncio

current_dir = os.getcwd()
# Append the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

# ignore warrnings
import warnings
warnings.filterwarnings("ignore")

#### Import Custom Scripts

In [2]:
# Importing custom scripts for image downloading and Telegram data scraping
from scripts.photo_scrapper import download_images
from scripts.telegram_data_scraper import scrape_messages

In [3]:
# Apply nest_asyncio to handle event loop issues in Jupyter
nest_asyncio.apply()

In [4]:
# Load environment variables from .env file
load_dotenv()

True

#### Fetch API Credentials from .env

In [5]:
# Fetch API credentials from the .env file using environment variables
api_id = os.getenv('API_ID')
api_hash = os.getenv('API_HASH')
phone_number = os.getenv('PHONE_NUMBER')

#### Configure Logging

In [6]:
# Set up logging to log messages to both a file and the console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("../logs/scraping.log"),  # Save logs to a file
        logging.StreamHandler()  # Output logs to console
    ]
)

In [7]:
# Initialize the Telegram client using environment variables
client = TelegramClient('session_name', api_id, api_hash)

#### Function to Scrape Messages from Channels

In [8]:
async def scrape_channels(channel_urls):
    """
    Scrapes images and messages from a list of Telegram channels.
    Args:
        channel_urls (list): List of Telegram channel URLs.
    Returns:
        final_df (DataFrame): Combined DataFrame containing messages from all channels.
    """
    all_messages = []

    for channel_url in channel_urls:
        # Scrape and download images
        # await scrape_images(channel_url)
        
        # Scrape messages and store in a DataFrame
        df = await scrape_messages(channel_url, client)
        all_messages.append(df)

    # Combine all DataFrames
    final_df = pd.concat(all_messages, ignore_index=True)
    return final_df

In [9]:
# Function to scrape images from a channel
async def scrape_images(channel_url, client):
    """
    Scrape images from a Telegram channel and download them.

    Args:
        channel_url: The URL of the Telegram channel.
        client: The Telegram client used to interact with the Telegram API.
    """
    try:
        logging.info(f"Scraping images from channel: {channel_url}")
        
        # Get the channel entity
        channel = await client.get_entity(channel_url)
        channel_name = channel.username or channel.title.replace(' ', '_')

        # Scrape messages and download images
        async for message in client.iter_messages(channel):
            await download_images(message, channel_name, client)
        
        logging.info(f"Finished scraping images from channel: {channel_url}")
    except Exception as e:
        logging.error(f"Error scraping images from channel {channel_url}: {e}")

In [10]:
async def main():
    """
    Asynchronous function to run the scraping process for multiple Telegram channels.
    
    Scrapes messages from the specified channels, saves the data to a CSV file, and returns the DataFrame.

    Returns:
        df (DataFrame): DataFrame containing scraped messages from all specified Telegram channels.
    """
    channel_urls = ['t.me/DoctorsET', 't.me/lobelia4cosmetics', 't.me/CheMed123', 't.me/EAHCI', 'yetenaweg']  
    try:
        # Start the Telegram client
        await client.start(phone=phone_number)
        
        # Scrape channels and get the final DataFrame
        df = await scrape_channels(channel_urls)
        
        # Save the DataFrame to a CSV file
        df.to_csv('../data/scraped_telegram_messages.csv', index=False)
        
        logging.info("Data successfully saved to '../data/scraped_telegram_messages.csv'")
        return df
    except Exception as e:
        logging.error(f"Error in main process: {e}")

#### Execute the Main Function

In [11]:
# Run the main function
await main()

2025-01-30 18:39:15,892 - INFO - Connecting to 149.154.167.91:443/TcpFull...
2025-01-30 18:39:16,032 - INFO - Connection to 149.154.167.91:443/TcpFull complete!
2025-01-30 18:39:16,954 - INFO - Scraping messages from channel: t.me/DoctorsET
2025-01-30 18:39:25,363 - INFO - Finished scraping messages from channel: t.me/DoctorsET
2025-01-30 18:39:25,365 - INFO - Scraping messages from channel: t.me/lobelia4cosmetics
2025-01-30 18:39:48,714 - INFO - Finished scraping messages from channel: t.me/lobelia4cosmetics
2025-01-30 18:39:48,715 - INFO - Scraping messages from channel: t.me/CheMed123
2025-01-30 18:39:49,331 - INFO - Finished scraping messages from channel: t.me/CheMed123
2025-01-30 18:39:49,332 - INFO - Scraping messages from channel: t.me/EAHCI
2025-01-30 18:40:11,956 - INFO - Finished scraping messages from channel: t.me/EAHCI
2025-01-30 18:40:11,958 - INFO - Scraping messages from channel: yetenaweg
2025-01-30 18:40:23,329 - INFO - Finished scraping messages from channel: yetena

,message_id,text,sender,channel,date
0,DoctorsET_864,https://youtu.be/5DBoEm-8kmA?si=LDLuEecNfULJVD...,-1001102021238,DoctorsET,2023-12-18 17:04:02+00:00
1,DoctorsET_863,ዶክተርስ ኢትዮጵያ በ አዲስ አቀራረብ በ ቴሌቪዥን ፕሮግራሙን ለመጀመር ከ...,-1001102021238,DoctorsET,2023-11-03 16:14:39+00:00
2,DoctorsET_862,ሞት በስኳር \n\nለልጆቻችን የምናሲዘው ምሳቃ ሳናቀው እድሚያቸውን ይቀን...,-1001102021238,DoctorsET,2023-10-02 16:37:39+00:00
3,DoctorsET_861,ከ HIV የተፈወሰ ሰው አጋጥሟችሁ ያቃል ? ፈውስ እና ህክምና ?\n\nሙ...,-1001102021238,DoctorsET,2023-09-16 07:54:32+00:00
4,DoctorsET_860,በቅርብ ጊዜ በሃገራችን ላይ እየተስተዋለ ያለ የተመሳሳይ ፆታ ( Homos...,-1001102021238,DoctorsET,2023-09-01 16:16:15+00:00
...,...,...,...,...,...
6475,yetenaweg_5,ይህ አዲሱ የኮሮና ቫይረስ በማይክሮስኮፕ ስር ሲታይ ያለው ምስል ነው። ኮ...,-1001447066276,yetenaweg,2020-02-17 20:58:59+00:00
6476,yetenaweg_4,አዲሱ የኮሮና ቫይረስ (በአዲስ የሳይንስ ስሙ COVID-19) ፣\nከየት ...,-1001447066276,yetenaweg,2020-02-17 20:55:46+00:00
6477,yetenaweg_3,,-1001447066276,yetenaweg,2020-02-17 20:55:05+00:00
6478,yetenaweg_2,,-1001447066276,yetenaweg,2020-01-17 01:35:09+00:00


### Extract Images from telegram channels

In [12]:
# Asynchronous main function to run the scraping process
async def main():
    channel_urls = ['t.me/lobelia4cosmetics', 't.me/CheMed123']
    try:
        # Start the Telegram client
        await client.start(phone=phone_number)
        
        # Scrape channels
        for channel_url in channel_urls:
            await scrape_images(channel_url, client)
    except Exception as e:
        logging.error(f"Error in main process: {e}")

In [13]:
await main()

2025-01-30 18:40:23,614 - INFO - Scraping images from channel: t.me/lobelia4cosmetics
2025-01-30 18:40:24,351 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2025-01-30 18:40:24,758 - INFO - Downloaded image from message 15192 in lobelia4cosmetics
2025-01-30 18:40:24,760 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2025-01-30 18:40:25,997 - INFO - Downloaded image from message 15191 in lobelia4cosmetics
2025-01-30 18:40:25,999 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2025-01-30 18:40:26,247 - INFO - Downloaded image from message 15190 in lobelia4cosmetics
2025-01-30 18:40:26,249 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2025-01-30 18:40:26,372 - INFO - Downloaded image from message 15189 in lobelia4cosmetics
2025-01-30 18:40:26,374 - INFO - Starting direct file download in chunks of 131072 at 0, stride 131072
2025-01-30 18:40:26,492 - INFO - Downlo

2025-01-30 18:48:05,874 - INFO - Got difference for account updates
2025-01-30 18:48:15,312 - INFO - Got difference for channel 2172718301 updates
2025-01-30 18:50:17,615 - INFO - Got difference for channel 1654774087 updates
2025-01-30 18:52:09,033 - INFO - Got difference for account updates
2025-01-30 18:52:09,330 - INFO - Got difference for channel 1764387820 updates
2025-01-30 18:52:09,534 - INFO - Got difference for channel 1416269677 updates
2025-01-30 18:52:09,739 - INFO - Got difference for channel 1243354741 updates
2025-01-30 18:52:09,943 - INFO - Got difference for channel 1243111354 updates
2025-01-30 18:52:10,148 - INFO - Got difference for channel 1341315103 updates
2025-01-30 18:54:17,835 - INFO - Got difference for channel 2172718301 updates
2025-01-30 18:57:43,345 - INFO - Got difference for channel 1678112693 updates
2025-01-30 18:59:34,341 - INFO - Got difference for channel 1390722118 updates
2025-01-30 18:59:34,541 - INFO - Got difference for channel 1764387820 upd